In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import spacy

In [2]:
analyzer = SentimentIntensityAnalyzer()
nlp = spacy.load('en')

In [3]:
test_sentence = 'Donald Trump is a terrible president. Bernie Sanders would be much better!'

In [4]:
doc = nlp(test_sentence)
subtokens = [token for token in doc] 

In [5]:
print([toc.dep_ for toc in subtokens])

['compound', 'nsubj', 'ROOT', 'det', 'amod', 'attr', 'punct', 'compound', 'nsubj', 'aux', 'ROOT', 'advmod', 'acomp', 'punct']


In [6]:
print(subtokens)

[Donald, Trump, is, a, terrible, president, ., Bernie, Sanders, would, be, much, better, !]


In [7]:
def analyze(sentence):
    subject_scores = {}
    doc = nlp(sentence)
    for token in doc:
        if token.dep_ == 'nsubj':
            scores = analyzer.polarity_scores(token.sent.text)
            score = scores['compound']
            subject_scores[token] = score * 5 + 5
    return subject_scores
    

In [8]:
analyze(test_sentence)

{Trump: 2.6165, Sanders: 7.463}